# **Segmentation Analysis**:<a class="anchor" id="0"></a>

1. [**Data Import**](#1)
2. [**Segmentation**](#2)

In [1]:
import numpy as np
import pandas as pd

from _util.custom_plotting import *
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

from _util.custom_mem_opt import custom_mem_opt

sns.set_style('darkgrid')
color = sns.color_palette()

root = './_data/'

In [2]:
### Data Import <a class=anchor id="1"></a>
[Back to top](#0)

SyntaxError: invalid syntax. Perhaps you forgot a comma? (1645189468.py, line 2)